In [66]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,ReLU,Add,AveragePooling2D,Dense,Flatten,Dropout

In [47]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    seed=123,
    batch_size=32,
    image_size=(224,224))


test_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/test/',
    seed=123,
    image_size=(224,224))

Found 7328 files belonging to 15 classes.
Found 1841 files belonging to 15 classes.


In [48]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [78]:
class ResNet34(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet34, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu1 = tf.keras.layers.Activation("relu")
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding="same")
        
        self.block1 = self.build_block(64, 3, first_block=True)
        self.block2 = self.build_block(128, 4)
        self.block3 = self.build_block(256, 6)
        self.block4 = self.build_block(512, 3)
        
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=num_classes, activation="softmax")
        
    def build_block(self, filters, num_blocks, first_block=False):
        strides = 1
        if not first_block:
            strides = 2
            
        layers = []
        layers.append(tf.keras.layers.Conv2D(filters=filters, kernel_size=1, strides=strides, padding="same"))
        layers.append(tf.keras.layers.BatchNormalization())
        layers.append(tf.keras.layers.Activation("relu"))
        
        for _ in range(num_blocks):
            layers.append(tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=1, padding="same"))
            layers.append(tf.keras.layers.BatchNormalization())
            layers.append(tf.keras.layers.Activation("relu"))
            
        return tf.keras.Sequential(layers)
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        
        x = self.avgpool(x)
        x = self.fc(x)
        
        return x

In [79]:
num_classes=15
model = ResNet34(num_classes)

In [80]:
model.build(input_shape=(32,224,224,3))

In [81]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [59]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [82]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [83]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [84]:
EPOCHS = 10
with tf.device('/device:GPU:0'):
    for epoch in range(EPOCHS):
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        for images, labels in train_ds:
            train_step(images, labels)
        for test_images, test_labels in test_ds:
            test_step(test_images, test_labels)
        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}')       

c:\Users\justf\miniconda3\envs\gpu\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 1, Loss: 2.6358394622802734, Accuracy: 14.287664413452148, Test Loss: 9.059215545654297, Test Accuracy: 8.039109230041504
Epoch 2, Loss: 2.4731245040893555, Accuracy: 19.978164672851562, Test Loss: 4.7100419998168945, Test Accuracy: 11.026616096496582
Epoch 3, Loss: 2.4356935024261475, Accuracy: 21.233623504638672, Test Loss: 7.826407432556152, Test Accuracy: 13.253666877746582
Epoch 4, Loss: 2.396591901779175, Accuracy: 22.502729415893555, Test Loss: 3.019244432449341, Test Accuracy: 17.110265731811523
Epoch 5, Loss: 2.3565561771392822, Accuracy: 23.21233558654785, Test Loss: 2.9512381553649902, Test Accuracy: 21.944595336914062
Epoch 6, Loss: 2.344533920288086, Accuracy: 23.66266441345215, Test Loss: 2.686108112335205, Test Accuracy: 21.292776107788086
Epoch 7, Loss: 2.3119351863861084, Accuracy: 24.945415496826172, Test Loss: 2.7766499519348145, Test Accuracy: 18.196632385253906
Epoch 8, Loss: 2.2485556602478027, Accuracy: 27.442684173583984, Test Loss: 2.458317756652832, Test